# How to calculate coherence lifetime of eddies

In this notebook I have collected the functions that are needed to go from ocean parcels trajectory dataset to **(1)** calculate LAVD cumulatevly for all time steps in the trajectory dataset, **(2)** identify RCLVs in all these LAVD fields.

The wast majority of functions are taken from or hevily inspired by RCLVatlas by Lexi Jones-Kellet from MIT.

In [2]:
import os,sys
import trajan
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import cmocean
from datetime import datetime
import pandas as pd
from shapely.geometry import Polygon
import geopandas as gpd
import pyproj

from config import *

sys.path.append('./RCLVatlas')
from subfunctions_for_RCLV_atlas import read_RCLV_CSV_untracked,read_RCLV_CSV_tracked,save_RCLV_CSV
from mainfunctions_for_RCLV_atlas import *
from functions_for_parcels import *
from scipy import integrate

## Input data 

Here we use ocean parcels .zarr dataset of 32 days as input. 

30x30 degree box around Hawaii

In [3]:
traj_path= "/work/bk1377/b382618/RCLVs/f32day/lang_traj/20100423_32days_runtime_20min_timestep_particle_start_lat_10.0_40.0_lon_180.0_210.0_spatial_step_0.03125_6hr_output_freq.zarr"

In [4]:
base_path = "/work/bk1377/b382618/LAVD_days_function/"

## Calculate LAVD for all days in the trajectory dataset

The function calculates LAVD for all trajectory time stepas in the dataset. For example, LAVD for day 7 out of 32 will integrate from time=0 to time=7 days. The original LAVD function (like in RCLV) usually integrate only the full trajectory lifetime. Here daily snapshots are available.


In [14]:
# a bit weirf function to deal with different date formats in different datasets
#this function is from chatgpt
def cleanup_nc_time_format(time_str):
    """
    Removes the last three zeros (microseconds) from a .nc file time string.
    
    Parameters:
    time_str (str): Time string in the format "2010-05-24T18:00:00.000000000"
    
    Returns:
    str: Time string with the last three zeros removed, in the format "2010-05-24T18:00:00"
    """
    if '.' in time_str:
        time_str = time_str[:time_str.index('.')]
    time_obj = datetime.datetime.strptime(time_str, "%Y-%m-%dT%H:%M:%S")
    
    return time_obj.strftime("%Y-%m-%dT%H:%M:%S")

In [15]:
# this is the real function, pls don't delete it
def calc_LAVD_spec_time(vort,output_freq,runtime, LAVD_days):
    time_select=int(LAVD_days*(24/output_freq))


    
    vort_avg_t = np.nanmean(vort,axis=0)[0:] #Find average vorticity over the entire spatial domain at each time step
    
    if LAVD_days==0:
        LAVD = np.absolute(vort[:,0]-vort_avg_t[0]) #this is just vorticity field
        # LAVD = np.trapezoid(np.absolute(vort[:,0:time_select] - vort_avg_t[0:time_select]), dx=output_freq*60*60, axis=1)/(time_select*6*60*60-output_freq*60*60) #trapz does the integration; convert data to be in se
        # LAVD = np.trapezoid(np.absolute(vort[:,0:time_select] - vort_avg_t[0:time_select]), dx=output_freq*60*60, axis=1) #trapz does the integration; convert data to be in se

    else:
        # LAVD = integrate.cumulative_trapezoid(np.absolute(vort[:,0:time_select] - vort_avg_t[0:time_select]), dx=output_freq*60*60, axis=1)/(LAVD_days*24*60*60-output_freq*60*60)
        ### if i do the above then the values at 0 are small and correct at the end.
        LVD = integrate.cumulative_trapezoid(np.absolute(vort[:,0:time_select] - vort_avg_t[0:time_select]), dx=output_freq*60*60, axis=1)   
        # LAVD = np.trapezoid(np.absolute(vort[:,0:time_select] - vort_avg_t[0:time_select]), dx=output_freq*60*60, axis=1)/(LAVD_days*24*60*60-output_freq*60*60) #trapz does the integration; convert data to be in se
    return LVD, time_select


In [16]:
def trajectories_to_specific_day_LAVD(traj_path, LAVD_days):
    print("Calculating the LAVD ...")
    traj_ds = xr.open_dataset(traj_path) # open the Lagrangian trajectory dataset that was just produced
    date = str(traj_ds.time.values[0][0])[0:10]
    vort_premask = traj_ds.variables["vort"]
    vort = np.array(vort_premask.where(vort_premask != 0)) #filters out land values
    LVD, time_select = calc_LAVD_spec_time(vort,sim_params['output_freq'],sim_params['runtime'], LAVD_days)


    # z=sim_params['output_freq']*60*60
    divide_by_list = []
    for i in np.arange(2, time_select+1):
        val = (i*sim_params['output_freq']*60*60-sim_params['output_freq']*60*60)
        divide_by_list.append(val)
    LAVD = LVD/divide_by_list

    # Create an xarray Dataset with the necessary dimensions
    one_lat = np.unique(traj_ds.lat[:,0])
    one_lon = np.unique(traj_ds.lon[:,0])
    time = traj_ds.time[0][1:time_select].values
    # cleaned_time_strings = np.vectorize(cleanup_nc_time_format)(time_strings)
    time2= np.vectorize(cleanup_nc_time_format)(time.astype(str))
    print(time2)

    # Check the shape of LAVD and reshape it    
    # Reshape LAVD from (921600, 17) to (17, 960, 960)
    num_lat = len(one_lat)
    num_lon = len(one_lon)
    num_time = len(time2)
    
    if LAVD.shape == (num_lat * num_lon, num_time):
        LAVD = np.reshape(LAVD, (num_lon, num_lat, num_time))
    else:
        raise ValueError(f"Unexpected LAVD shape {LAVD.shape}. Cannot reshape.")
    
    LAVD_ds = xr.Dataset(
        {
            'LAVD': (['longitude', 'latitude', 'time'], LAVD)
        },
        coords={
            'time': time2,
            'latitude': one_lat,
            'longitude': one_lon
        }
    )

    date_good=str(date.replace("-", ""))
    output_file_name = str(date_good) + "_" + str(LAVD_days) + "days_LAVD.nc"
    # Save the dataset as a NetCDF file
    LAVD_output_file_path = os.path.join(base_path, output_file_name)
    LAVD_ds.to_netcdf(LAVD_output_file_path)
    
    print('LAVD NetCDF output file: %s' % (LAVD_output_file_path))
    return LAVD_output_file_path


In [17]:
out_p = trajectories_to_specific_day_LAVD(traj_path, 32)

Calculating the LAVD ...


/home/b/b382618/.conda/envs/parcels2/lib/python3.11/site-packages/xarray/backends/plugins.py:159: RuntimeWarning: 'netcdf4' fails while guessing
  warnings.warn(f"{engine!r} fails while guessing", RuntimeWarning)
/home/b/b382618/.conda/envs/parcels2/lib/python3.11/site-packages/xarray/backends/plugins.py:159: RuntimeWarning: 'scipy' fails while guessing
  warnings.warn(f"{engine!r} fails while guessing", RuntimeWarning)


['2010-04-23T06:00:00' '2010-04-23T12:00:00' '2010-04-23T18:00:00'
 '2010-04-24T00:00:00' '2010-04-24T06:00:00' '2010-04-24T12:00:00'
 '2010-04-24T18:00:00' '2010-04-25T00:00:00' '2010-04-25T06:00:00'
 '2010-04-25T12:00:00' '2010-04-25T18:00:00' '2010-04-26T00:00:00'
 '2010-04-26T06:00:00' '2010-04-26T12:00:00' '2010-04-26T18:00:00'
 '2010-04-27T00:00:00' '2010-04-27T06:00:00' '2010-04-27T12:00:00'
 '2010-04-27T18:00:00' '2010-04-28T00:00:00' '2010-04-28T06:00:00'
 '2010-04-28T12:00:00' '2010-04-28T18:00:00' '2010-04-29T00:00:00'
 '2010-04-29T06:00:00' '2010-04-29T12:00:00' '2010-04-29T18:00:00'
 '2010-04-30T00:00:00' '2010-04-30T06:00:00' '2010-04-30T12:00:00'
 '2010-04-30T18:00:00' '2010-05-01T00:00:00' '2010-05-01T06:00:00'
 '2010-05-01T12:00:00' '2010-05-01T18:00:00' '2010-05-02T00:00:00'
 '2010-05-02T06:00:00' '2010-05-02T12:00:00' '2010-05-02T18:00:00'
 '2010-05-03T00:00:00' '2010-05-03T06:00:00' '2010-05-03T12:00:00'
 '2010-05-03T18:00:00' '2010-05-04T00:00:00' '2010-05-04T06:00

## Identify RCLVs in the created LAVD daily fields - check if it saves better now with temp = []

In [18]:
nc_file_path = "/work/bk1377/b382618/LAVD_days_function/20100423_32days_LAVD.nc"

In [19]:
traj_path= "/work/bk1377/b382618/RCLVs/f32day/lang_traj/20100423_32days_runtime_20min_timestep_particle_start_lat_10.0_40.0_lon_180.0_210.0_spatial_step_0.03125_6hr_output_freq.zarr"

In [20]:
def extract_particles_after_time_for_LAVD(traj,x_mask,y_mask,traj_lat_array,sim_params,obs_to_select):
    """
    Get the lat/lons of particles from a ploygon after some number of days along the Lagrangian trajectory.
    
    Input
        traj: trajectory file 
        x_mask,y_mask: indeces from the longitude & latitude arrays that are inside of the polygon
        days: number of days from the initialization time to retreive particle locations (back trajectories will be back in time)
    Output
        eddy_xdays_lons,eddy_xday_lats: lon/lat coordinates of the particles of interest on day x
    
    """
    particle_lon,particle_lat,particle_vort = traj.variables["lon"],traj.variables["lat"],traj.variables["vort"] #read in particle location lat, lons, vorts
    particle_nums = x_mask*len(traj_lat_array) + y_mask #formula to get the integer ID of the particles inside a polygon
    if obs_to_select==0: ### not sure if this if statement is needed
        eddy_xday_lons = [float(particle_lon[p,int(((24/sim_params['output_freq'])*obs_to_select))]) for p in particle_nums]   
        eddy_xday_lats = [float(particle_lat[p,int(((24/sim_params['output_freq'])*obs_to_select))]) for p in particle_nums]
        eddy_xday_vorts = [float(particle_vort[p,int(((24/sim_params['output_freq'])*obs_to_select))]) for p in particle_nums]

    else:
######Stella- added -1 to the int bc i though that the indexing does not work bc of that. i think it should be like that because it is indexing         
        eddy_xday_lons = [float(particle_lon[p,int(obs_to_select)]) for p in particle_nums]   
        eddy_xday_lats = [float(particle_lat[p,int(obs_to_select)]) for p in particle_nums]
        eddy_xday_vorts = [float(particle_vort[p,int(obs_to_select)]) for p in particle_nums]
    return eddy_xday_lons,eddy_xday_lats,eddy_xday_vorts

In [35]:
def LAVD_days_to_RCLV_loop(traj_ds_path, LAVD_ds_path):
    RCLV_data = [['date', 'RCLV_id', 'orientation', 'age_days', 'area_km2', 'center_lon', 'center_lat', 'CD', 'flag', 'boundary_coords']]

    traj_ds = xr.open_zarr(traj_ds_path)
    lavd_ds = xr.open_dataset(LAVD_ds_path)
    lavd_ds = lavd_ds.LAVD
    
    time_list = traj_ds.time[0].values
    midnight_list = time_list[::4]

    # midnight_list = midnight_list[9:]
    # midnight_list = midnight_list[1:] #this removes the first date when particles start, so the first RCLVS will be identified at 1 day later
    print(midnight_list)

    # ## now i need to itterate the midnight list
    # plms_cum,cons_cum,areas_cum,cds_cum,pols_cum = [],[],[],[],[]
    for sk in np.arange(1, len(midnight_list)):
        ## select the right LAVD field
        right_date = midnight_list[sk]
        print(right_date)
        short_date=str(right_date)[0:19]
        print(short_date)
        lavd_subselect = lavd_ds.sel(time=short_date)
        #select subselect the lavd field 
        traj_subselect = traj_ds.isel(obs = slice(0, sk*4))
        
        # LAVD = np.load('%s%s_LAVD_%s.npy'%(LAVD_dir,initial_date,filename_str)) #Load LAVD data
        LAVD = np.ma.masked_where(np.isnan(lavd_subselect),lavd_subselect) #Land mask required for the peak_local_max function to work
        LAVD_reshape = np.transpose(np.reshape(LAVD,(len(traj_lon_array),len(traj_lat_array))))
        
        # Iterate through all of the local maxima to find structures that meet our RCLV criteria 
        plms,cons,areas,cds,pols = [],[],[],[],[] # arrays to store the output
        plm = peak_local_max(LAVD_reshape,min_distance=RCLV_params['min_dist']) # grid indices of the local maxima in the LAVD field
        for ji in plm:
            args = {'lon0':traj_lon_array[ji[1]],
                    'lat0':traj_lat_array[ji[0]],
                    'dlon':np.abs(traj_lon_array[1]-traj_lon_array[0]),
                    'dlat':np.abs(traj_lat_array[1]-traj_lat_array[0])}
            
            for target_CD in [0.03,0.02,0.01]: #target CDs to test, will iterate through each of these unless if a successful contour is identified
                con,area,cd = rclv.convex_contour_around_maximum(LAVD_reshape,ji,
                                                                 init_contour_step_frac=RCLV_params['init_contour_step_frac'],
                                                                 convex_def_tol=RCLV_params['convex_def_tol'],
                                                                 convex_def=target_CD)
            
                try: # Code errors if the local maximum is too close to the land where there are missing values
                    if (area >= RCLV_params['min_area']) and (cd <= 0.03): # check area and convex deficiency meet thresholds; actual cd identified will be different from target CD
                        lon_inds = [round(i) for i in con[:,1]]
                        lat_inds = [round(i) for i in con[:,0]]
    
                        # Reformat points of the polygon with parentheses
                        poly_pts = [(traj_lon_array[lon_inds][pt],traj_lat_array[lat_inds][pt]) for pt in np.arange(0,len(lon_inds))]
    
                        # Get the CI of the particles after simulation run time to make sure it meets threshold set
                        x_mask,y_mask = find_polygon_pts(poly_pts,traj_lon_array,traj_lat_array)                    
                        eddy_day0_lons,eddy_day0_lats,_ = extract_particles_after_time_for_LAVD(traj_subselect,x_mask,y_mask,traj_lat_array,sim_params,0) #day 0 data
                        eddy_dayx_lons,eddy_dayx_lats,eddy_dayx_vorts = extract_particles_after_time_for_LAVD(traj_subselect,x_mask,y_mask,traj_lat_array,sim_params,sk) # last day
                        contour_CI = CI(eddy_day0_lons,eddy_day0_lats,eddy_dayx_lons,eddy_dayx_lats) #calculate the coherency index (measure of dispersal)
    
                        if (contour_CI >= -0.5): # Coherency index criteria, minimal dispersal required
                            # Check that the vorticty is consistent within the vortex over the course of the simulation (filters out some weird saddle point features,etc)
                            # Timestep to get initial vorticity needs to be day 1 since at day 0 all particles are initialized with vort=0
                            eddy_day1_lons,eddy_day1_lats,eddy_day1_vorts = extract_particles_after_time(traj_subselect,x_mask,y_mask,traj_lat_array,sim_params,1) 
                            day1_vort_signs = [np.sign(v) for v in eddy_day1_vorts]
                            day1_anti_count,day1_cyc_count = day1_vort_signs.count(-1.0),day1_vort_signs.count(1.0)  
                            day1_vort_freq = np.max((day1_anti_count,day1_cyc_count))/(day1_anti_count+day1_cyc_count) #percentage of particles with the same vorticity
    
                            if (day1_vort_freq >= 0.85): # 85% or more have the same vort sign on day 1
                                dayx_vort_signs = [np.sign(v) for v in eddy_dayx_vorts]
                                dayx_anti_count,dayx_cyc_count = dayx_vort_signs.count(-1.0),dayx_vort_signs.count(1.0)  
                                dayx_vort_freq = np.max((dayx_anti_count,dayx_cyc_count))/(dayx_anti_count+dayx_cyc_count)
    
                                if day1_anti_count > day1_cyc_count: #get the polarity of the vortex on day 1
                                    day1_vort_pol = 'anti'
                                else:
                                    day1_vort_pol = 'cyc'
                                if dayx_anti_count > dayx_cyc_count: #get the polarity of the vortex on last day
                                    dayx_vort_pol = 'anti'
                                else:
                                    dayx_vort_pol = 'cyc'

                                # If all criteria are passed, add values to output arrays & move on to next local maxima
                                if (dayx_vort_freq >= 0.85) and (day1_vort_pol == dayx_vort_pol): # Final criteria: vort is consistent for 85% of particles on the last day
                                    region_area = rclv.polygon_area(rclv.project_vertices(con, **args)) #Convert area units from pixels to m^2
                                    plms.append(ji)
                                    cons.append(con)
                                    areas.append(region_area/(10**6)) #convert from m^2 -> km^2
                                    cds.append(cd)
                                    pols.append(day1_vort_pol)
                                    break # do not need to look at other CDs if the criteria to this point has been met 
                except TypeError: # too close to land to define an RCLV
                    break
#### save the rclvs in a file
            for i in np.arange(0,len(cons)):
                temp = [short_date,np.nan,pols[i],np.nan,areas[i],traj_lon_array[plms[i][1]],traj_lat_array[plms[i][0]],cds[i],0] 
                
                # Convert from grid indices to lat/lon coordinates for the output
                lon_bounds = traj_lon_array[[round(j) for j in cons[i][:, 1]]]
                lat_bounds = traj_lat_array[[round(j) for j in cons[i][:, 0]]]
                for j in np.arange(0,len(lon_bounds)):
                    temp.extend([lon_bounds[j],lat_bounds[j]])
                RCLV_data.append(temp) # Add RCLV data to main array
        temp = []
    return RCLV_data

In [36]:
output_data = LAVD_days_to_RCLV_loop(traj_path, nc_file_path)


['2010-05-02T00:00:00.000000000' '2010-05-03T00:00:00.000000000'
 '2010-05-04T00:00:00.000000000' '2010-05-05T00:00:00.000000000'
 '2010-05-06T00:00:00.000000000' '2010-05-07T00:00:00.000000000'
 '2010-05-08T00:00:00.000000000' '2010-05-09T00:00:00.000000000'
 '2010-05-10T00:00:00.000000000' '2010-05-11T00:00:00.000000000'
 '2010-05-12T00:00:00.000000000' '2010-05-13T00:00:00.000000000'
 '2010-05-14T00:00:00.000000000' '2010-05-15T00:00:00.000000000'
 '2010-05-16T00:00:00.000000000' '2010-05-17T00:00:00.000000000'
 '2010-05-18T00:00:00.000000000' '2010-05-19T00:00:00.000000000'
 '2010-05-20T00:00:00.000000000' '2010-05-21T00:00:00.000000000'
 '2010-05-22T00:00:00.000000000' '2010-05-23T00:00:00.000000000'
 '2010-05-24T00:00:00.000000000']
2010-05-03T00:00:00.000000000
2010-05-03T00:00:00
variance_t0:  181.56661476246995 variance_tx: 196.99375354727567
variance_t0:  3604.366225899661 variance_tx: 3627.4801500484386
variance_t0:  1134.3912917019165 variance_tx: 1129.9771402505467
varianc

In [37]:
directory_of_LAVD_days = nc_file_path.rsplit('/', 1)[0]

In [38]:
save_RCLV_CSV(output_data, directory_of_LAVD_days + '32_day_RCLV_f10_end.csv') #save RCLV as CSV

### I sould add dates from midnight list to the saving path

In [ ]:
# save_RCLV_CSV(output_data,'/home/b/b382618/try_rclv_fun/32_day_LAVD_eddies.csv') #save RCLV as CSV

## post-processing daily RCLVs

In [5]:
path = '/work/bk1377/b382618/LAVD_days_function32_day_RCLV_f0_10.csv'

In [6]:
path2 = '/work/bk1377/b382618/LAVD_days_function32_day_RCLV_f10_end.csv'

In [7]:
nc_file_path = "/work/bk1377/b382618/LAVD_days_function/20100423_32days_LAVD.nc"

In [3]:
# path = "/home/b/b382618/try_rclv_fun/32_day_LAVD_eddies.csv" 

Open csv file as a pandas df

In [8]:
### 
# Step 1: Open and process the file
processed_data = []
with open(path, 'r') as file:
    for line in file:
        # Split the line into entries and process
        row = line.strip().split(',')
        first_9 = row[:9]  # First 9 entries
        rest = row[9:]     # Remaining entries
        processed_data.append(first_9 + [rest])  # Combine first 9 and rest

# Step 4: Create DataFrame with transformed data
columns = [f"Col_{i+1}" for i in range(9)] + ['Extra_Entries']
df = pd.DataFrame(processed_data, columns=columns)

# Set first row as column headers
df.columns = df.iloc[0].astype(str)
df = df[1:].reset_index(drop=True)
df.columns.values[-1] = 'Points'


In [9]:
### open the other ds
### 
# Step 1: Open and process the file
processed_data = []
with open(path2, 'r') as file:
    for line in file:
        # Split the line into entries and process
        row = line.strip().split(',')
        first_9 = row[:9]  # First 9 entries
        rest = row[9:]     # Remaining entries
        processed_data.append(first_9 + [rest])  # Combine first 9 and rest

# Step 4: Create DataFrame with transformed data
columns = [f"Col_{i+1}" for i in range(9)] + ['Extra_Entries']
df2 = pd.DataFrame(processed_data, columns=columns)

# Set first row as column headers
df2.columns = df2.iloc[0].astype(str)
df2 = df2[1:].reset_index(drop=True)
df2.columns.values[-1] = 'Points'


In [10]:
## join the 2 dfs

result = pd.concat([df, df2], ignore_index=True)


In [11]:
result

,date,RCLV_id,orientation,age_days,area_km2,center_lon,center_lat,CD,flag,Points
0,2010-04-24T00:00:00,nan,cyc,nan,1220.3238762319756,197.21875,22.59375,0.02917352572318881,0,"[197.25, 22.75, 197.25, 22.75, 197.28125, 22.7..."
1,2010-04-24T00:00:00,nan,cyc,nan,1220.3238762319756,197.21875,22.59375,0.02917352572318881,0,"[197.25, 22.75, 197.25, 22.75, 197.28125, 22.7..."
2,2010-04-24T00:00:00,nan,cyc,nan,1220.3238762319756,197.21875,22.59375,0.02917352572318881,0,"[197.25, 22.75, 197.25, 22.75, 197.28125, 22.7..."
3,2010-04-24T00:00:00,nan,cyc,nan,1220.3238762319756,197.21875,22.59375,0.02917352572318881,0,"[197.25, 22.75, 197.25, 22.75, 197.28125, 22.7..."
4,2010-04-24T00:00:00,nan,cyc,nan,1220.3238762319756,197.21875,22.59375,0.02917352572318881,0,"[197.25, 22.75, 197.25, 22.75, 197.28125, 22.7..."
...,...,...,...,...,...,...,...,...,...,...
66925,2010-05-24T00:00:00,nan,anti,nan,1147.4436825425983,201.4375,32.59375,0.02605394582965364,0,"[201.59375, 32.78125, 201.59375, 32.78125, 201..."
66926,2010-05-24T00:00:00,nan,anti,nan,5766.380146460185,200.125,17.6875,0.019223482029749,0,"[200.46875, 17.90625, 200.46875, 17.90625, 200..."
66927,2010-05-24T00:00:00,nan,anti,nan,3127.1111104979846,193.875,16.53125,0.02073359760479094,0,"[193.78125, 16.78125, 193.8125, 16.78125, 193...."
66928,2010-05-24T00:00:00,nan,cyc,nan,1394.726033821938,196.46875,20.59375,0.020967693843752074,0,"[196.5625, 20.6875, 196.59375, 20.6875, 196.62..."


In [12]:
df=result

In [13]:
### open lavd file

lavd = xr.open_dataset(nc_file_path)

### fix datasets

In [14]:

def points_to_polygon(points_list):
    # Organize as coordinate pairs
    coordinates = [(points_list[i], points_list[i+1]) for i in range(0, len(points_list), 2)]
    # Close the polygon if needed
    if coordinates[0] != coordinates[-1]:
        coordinates.append(coordinates[0])
    return Polygon(coordinates)

In [15]:
df['Polygon'] = df['Points'].apply(points_to_polygon)


In [16]:
### i currently need to drop duplicates bc my function did not clear the temporary list and soo many of the same eddies are saved
df = df.applymap(lambda x: str(x) if isinstance(x, list) else x) #because there is a list of polygins
# Then remove duplicates
df = df.drop_duplicates()

/tmp/ipykernel_3875030/127318340.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: str(x) if isinstance(x, list) else x) #because there is a list of polygins


In [17]:
### save as a csv or pandas

In [18]:
type(df)

pandas.core.frame.DataFrame

In [20]:
df.to_pickle('/work/bk1377/b382618/LAVD_days_32_days_RCLVs_full.pickle')

In [21]:
df = pd.read_pickle('/work/bk1377/b382618/LAVD_days_32_days_RCLVs_full.pickle')

## Add group_id and age order for overlapping eddies

In [59]:
# from shapely.geometry import Polygon
# from shapely.strtree import STRtree
# import networkx as nx

# not using these but should

In [22]:
group_ids = [-1] * len(df)
current_id = 0

# Iterate through polygons to assign group IDs
for i, poly1 in enumerate(df["Polygon"]):
    if group_ids[i] == -1:  # If not yet assigned, create a new group
        group_ids[i] = current_id
        for j, poly2 in enumerate(df["Polygon"]):
            if i != j and poly1.intersects(poly2):  # Check for overlap
                group_ids[j] = current_id
        current_id += 1

# Add group IDs to the DataFrame
df.loc[:, "group_id"] = group_ids

In [23]:
df['date'] = pd.to_datetime(df['date'])
# Rank polygons within each group based on the 'Date' column
df['age_order'] = df.groupby('group_id')["date"].rank(method='first', ascending=True).astype(int) ## ascending=False for it being backwards trajectories

In [24]:
df.to_pickle('/work/bk1377/b382618/LAVD_days_32_days_RCLVs_full.pickle')

In [25]:
## All that was below is moved to coherence_lifetime_eddy_analysis